In [5]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

def load_and_process_results(csv_path):
    """Load and process the stress test results."""
    df = pd.read_csv(csv_path)
    # Only include successful runs
    df = df[df['success'] == True]
    return df

def create_visualizations(df):
    """Create various visualizations of the results."""
    # Create subplots for different metrics
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            'Execution Time vs Graph Size', 
            'Algorithm Metric vs Graph Size',
            'Success Rate vs Graph Size'
        ),
        vertical_spacing=0.15
    )

    # Color scheme for different algorithm types
    colors = {
        'shortest_path': '#1f77b4',  # Blue
        'clustering': '#2ca02c',     # Green
        'ranking': '#ff7f0e'         # Orange
    }

    # Plot execution time
    for algo_type in df['algorithm_type'].unique():
        data = df[df['algorithm_type'] == algo_type]
        fig.add_trace(
            go.Scatter(
                x=data['nodes'],
                y=data['execution_time'],
                name=f'{algo_type.replace("_", " ").title()}',
                line=dict(color=colors[algo_type]),
                mode='lines+markers'
            ),
            row=1, col=1
        )

    # Plot algorithm-specific metrics
    for algo_type in df['algorithm_type'].unique():
        data = df[df['algorithm_type'] == algo_type]
        fig.add_trace(
            go.Scatter(
                x=data['nodes'],
                y=data['metric'],
                name=f'{algo_type.replace("_", " ").title()} Metric',
                line=dict(color=colors[algo_type], dash='dash'),
                mode='lines+markers',
                showlegend=False
            ),
            row=2, col=1
        )

    # Plot success rate
    for algo_type in df['algorithm_type'].unique():
        data = df[df['algorithm_type'] == algo_type]
        success_rate = data.groupby('nodes')['success'].mean() * 100
        fig.add_trace(
            go.Scatter(
                x=success_rate.index,
                y=success_rate.values,
                name=f'{algo_type.replace("_", " ").title()} Success',
                line=dict(color=colors[algo_type], dash='dot'),
                mode='lines+markers',
                showlegend=False
            ),
            row=3, col=1
        )

    # Update layout
    fig.update_layout(
        title='Algorithm Performance Analysis',
        height=1000,
        width=1000,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    # Update axes labels
    for i in range(1, 4):
        fig.update_xaxes(title_text="Number of Nodes", row=i, col=1)
    
    fig.update_yaxes(title_text="Execution Time (seconds)", row=1, col=1)
    fig.update_yaxes(title_text="Algorithm Metric", row=2, col=1)
    fig.update_yaxes(title_text="Success Rate (%)", row=3, col=1)

    return fig

# Example usage in notebook:
# Load the results
results_path = 'results/stress_test_results.csv'  # Relative to the notebook location

df = load_and_process_results(results_path)

# Create and display the visualization
fig = create_visualizations(df)
fig.show()

# Optional: Save the plot
# fig.write_html('test/results/performance_analysis.html')

KeyError: 'success'